# 🏥 Módulo 8 – Batch ETL con PySpark (Google Colab)

## 1) Instalar y configurar Spark en Colab

In [1]:
!apt-get install -qq openjdk-11-jdk > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.7/spark-3.5.7-bin-hadoop3.tgz
!tar xf spark-3.5.7-bin-hadoop3.tgz
!pip -q install pyspark findspark

In [2]:
import os
os.environ['JAVA_HOME']='/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.5.7-bin-hadoop3'
print('JAVA_HOME =', os.environ['JAVA_HOME'])
print('SPARK_HOME =', os.environ['SPARK_HOME'])

JAVA_HOME = /usr/lib/jvm/java-11-openjdk-amd64
SPARK_HOME = /content/spark-3.5.7-bin-hadoop3


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Mod8-Batch-ETL').getOrCreate()
print('✅ Spark version:', spark.version)

✅ Spark version: 3.5.7


## 2) Cargar CSV hospitalario

In [4]:
HOSPITAL_CSV = '/content/hospital_santiago_280.csv'
df = spark.read.csv(HOSPITAL_CSV, header=True, inferSchema=True)
print('Filas:', df.count())
df.printSchema()
df.show(5, truncate=False)

Filas: 280
root
 |-- paciente_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- apellido: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- comuna: string (nullable = true)
 |-- servicio_clinico: string (nullable = true)
 |-- fecha_ingreso: date (nullable = true)
 |-- diagnostico_principal: string (nullable = true)
 |-- dias_hospitalizacion: integer (nullable = true)
 |-- costo_total: integer (nullable = true)
 |-- medico_tratante: string (nullable = true)
 |-- estado_alta: string (nullable = true)

+-----------+------+--------+----+----+-----------+----------------+-------------+-------------------------+--------------------+-----------+---------------+-----------+
|paciente_id|nombre|apellido|edad|sexo|comuna     |servicio_clinico|fecha_ingreso|diagnostico_principal    |dias_hospitalizacion|costo_total|medico_tratante|estado_alta|
+-----------+------+--------+----+----+-----------+----------------+-----------

## 3) Limpieza y transformación (básica)

In [5]:
from pyspark.sql.functions import col
NUM_TYPES={'int','bigint','double','float','long'}
df = df.toDF(*[c.strip().lower().replace(' ','_') for c in df.columns])
df = df.dropDuplicates()
num_cols=[c for (c,t) in df.dtypes if t in NUM_TYPES]
cat_cols=[c for (c,t) in df.dtypes if t not in NUM_TYPES]
for c in num_cols: df = df.na.fill({c:0})
for c in cat_cols: df = df.na.fill({c:'desconocido'})
print('✅ Limpieza básica completa')

✅ Limpieza básica completa


## 4) Consultas SQL (ejemplos)

In [6]:
df.createOrReplaceTempView('hospital')
try:
    spark.sql('SELECT diagnostico_principal, COUNT(*) AS total FROM hospital GROUP BY diagnostico_principal ORDER BY total DESC LIMIT 10').show()
except Exception as e:
    print('Ajusta columna diagnostico_principal. Error:', e)
try:
    spark.sql('SELECT servicio_clinico, ROUND(AVG(dias_hospitalizacion),2) AS estancia_promedio FROM hospital GROUP BY servicio_clinico ORDER BY estancia_promedio DESC').show()
except Exception as e:
    print('Ajusta servicio_clinico/dias_hospitalizacion. Error:', e)

+---------------------+-----+
|diagnostico_principal|total|
+---------------------+-----+
|  Enfermedad de Crohn|    2|
| Hepatitis autoinmune|    2|
|  Artritis reumatoide|    2|
| Insuficiencia car...|    2|
|    Lupus eritematoso|    2|
| Síndrome ovario p...|    2|
|  Síndrome de Sjögren|    2|
| Síndrome de Goodp...|    2|
| Esclerosis sistémica|    1|
|    Fractura de radio|    1|
+---------------------+-----+

+-----------------+-----------------+
| servicio_clinico|estancia_promedio|
+-----------------+-----------------+
|        Oncología|            16.13|
|       Neurología|            11.03|
|      Cardiología|            10.09|
|Gastroenterología|             9.64|
|    Traumatología|             7.34|
|         Urología|             7.17|
| Medicina Interna|              7.1|
|      Ginecología|             4.68|
|     Dermatología|             4.48|
+-----------------+-----------------+



## 5) Regresión simple (MLlib)

In [7]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
label_col='dias_hospitalizacion'
feature_exclude={label_col, 'fecha_ingreso'}
num_cols=[c for (c,t) in df.dtypes if t in NUM_TYPES and c not in feature_exclude]
cat_cols=[c for (c,t) in df.dtypes if t not in NUM_TYPES and c not in feature_exclude]
indexers=[StringIndexer(inputCol=c, outputCol=f'{c}_idx', handleInvalid='keep') for c in cat_cols]
encoder=OneHotEncoder(inputCols=[f'{c}_idx' for c in cat_cols], outputCols=[f'{c}_ohe' for c in cat_cols], handleInvalid='keep')
assembler=VectorAssembler(inputCols=num_cols+[f'{c}_ohe' for c in cat_cols], outputCol='features_raw')
scaler=StandardScaler(inputCol='features_raw', outputCol='features')
lr=LinearRegression(featuresCol='features', labelCol=label_col)
pipe=Pipeline(stages=indexers+[encoder,assembler,scaler,lr])
train,test=df.dropna(subset=[label_col]).randomSplit([0.8,0.2],seed=42)
model=pipe.fit(train)
pred=model.transform(test)
ev1=RegressionEvaluator(labelCol=label_col, predictionCol='prediction', metricName='rmse')
ev2=RegressionEvaluator(labelCol=label_col, predictionCol='prediction', metricName='r2')
print('RMSE:', round(ev1.evaluate(pred),4),'| R2:', round(ev2.evaluate(pred),4))

RMSE: 2.5829 | R2: 0.7439


## 6) Guardar artefactos y modelo

In [8]:
import json, os
os.makedirs('/content/artifacts', exist_ok=True)
os.makedirs('/content/models', exist_ok=True)
from pyspark.ml.evaluation import RegressionEvaluator
ev_rmse=RegressionEvaluator(labelCol='dias_hospitalizacion', predictionCol='prediction', metricName='rmse').evaluate(pred)
ev_r2=RegressionEvaluator(labelCol='dias_hospitalizacion', predictionCol='prediction', metricName='r2').evaluate(pred)
with open('/content/artifacts/batch_reg_metrics.json','w') as f:
    json.dump({'rmse': float(ev_rmse), 'r2': float(ev_r2)}, f, indent=2)
model.write().overwrite().save('/content/models/batch_lr_model')
print('✅ Guardados artefactos y modelo')

✅ Guardados artefactos y modelo
